In [ ]:
# colab google에서 연결된 런타임을 T4 GPU로 설정 후 실행해야함.
# install package and laod the extension
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm

# 터미널 내에 입력 (두줄 따로 입력)
"""
curl -fsSL https://ollama.com/install.sh | sh 
ollama serve & ollama pull llama3
"""

In [ ]:
# !pip install langchain
# !pip install langchain-core
# !pip install langchain-community
# !pip install langchain-ollama
poetry add langchain langchain-core langchain-community

In [ ]:
from langchain_community.llms import Ollama
# from langchain_ollama.llms import Ollama
llm = Ollama(model = "llama3")
llm.invoke("Tell me 3 red flower names")

In [ ]:
# !pip install -q streamlit
# !pip install streamlit_chat
# !pip install langchain
# !pip install langchain-core
# !pip install langchain-community
# !pip install chromadb
# !pip install pypdf
# !pip install fastembed
# poetry add -q streamlit
poetry add streamlit@1.11.1
poetry add transformers@4.46.0 # chromadb tokenizer 때문에 다운그레이드 
poetry add chromadb 
poetry add streamlit_chat langchain langchain-core langchain-community pypdf
[tool.poetry.dependencies]
python = ">=3.8,<3.13"
poetry add fastembed

In [2]:
%%writefile rag.py

from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.vectorstores.utils import filter_complex_metadata

class ChatPDF:
    vector_store = None
    retriever = None
    chain = None
    
    def __init__(self):
        self.model = ChatOllama(model="llama3")
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
        self.prompt = PromptTemplate.from_template(
            """
            <s> [INST] you are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
            If you don't konw the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. [/INST] </s>
            [INST] Question: {question}
            Context: {context}
            Answer: [/INST]
            """
        )

    def ingest(self, pdf_file_path: str):
        docs = PyPDFLoader(file_path=pdf_file_path).load()
        chunks = self.text_splitter.split_documents(docs)
        chunks = filter_complex_metadata(chunks)

        vector_store = Chroma.from_documents(documents=chunks, embedding=FastEmbedEmbeddings())
        self.retriever = vector_store.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={
                "k": 3,
                "score_threshold": 0.5
            },
        )

        self.chain = (
            {"context": self.retriever, "question": RunnablePassthrough()}
            | self.prompt
            | self.model
            | StrOutputParser()
        
        )

    def ask(self, query: str):
        if not self.chain:
            return "Please, add a PDF document first."
        
        return self.chain.invoke(query)

    def clear(self):
        self.vector_store = None
        self.retriever = None
        self.chain = None


        


Overwriting rag.py


In [3]:
%%writefile app.py
import os
import tempfile
import streamlit as st
from streamlit_chat import message
from rag import ChatPDF

st.set_page_config(page_title="ChatPDF")

def display_messages():
    st.subheader("Chat")
    for i, (msg, is_user) in enumerate(st.session_state["message"]):
        message(msg, is_user=is_user, key=str(i))

    st.session_state["thinking_spinner"] = st.empty()

def process_input():
    if st.session_state["user_input"] and len(st.session_state["user_input"].strip()) > 0:
        user_text= st.session_state["user_input"].strip()
        with st.session_state["thinking_spinner"], st.spinner(f"Thinking"):
            agent_text = st.session_state["assistant"].ask(user_text)

        st.session_state["messages"].append((user_text, True))
        st.session_state["messages"].append((agent_text, False))

# pdf 업로드시 저장
def read_and_save_file():
    st.session_state["assistant"].clear()
    st.session_state["messages"] = []
    st.session_state["user_input"] = ""

    for file in st.session_state["file_uploader"]:
        with tempfile.NamedTemporaryFile(delete=False) as tf:
            tf.write(file.getbuffer())
            file_path = tf.name

        with st.session_state["ingestion_spinner"], st.spinner(f"Ingesting {file.name}"):
            st.session_state["assistant"].ingest(file_path)
        os.remove(file_path)


def page():
    if len(st.session_state) == 0:
        st.session_state["messages"] = []
        st.session_state["assistant"] = ChatPDF()


    st.header("ChatPDF")

    st.subheader("Upload a document")
    st.file_uploader(
        "Upload document",
        type=["pdf"],
        key="file_uploader",
        on_change=read_and_save_file,
        label_visibility="collapsed",
        accept_multiple_files=True,
    )

    st.session_state["ingestion_spinner"] = st.empty()

    display_messages()
    st.text_input("Message", key="user_input", on_change=process_input)

    
if __name__ == "__main__":
    page()
    print("a")

Overwriting app.py


In [8]:
# 윈도우
# !streamlit run /content/app.py &>/content/logs.txt &
# !npx localtunnel --port 8501

# 맥 - 터미널
# 생성된 logs.txt에서 externalURL 34.169.66.99를 복사해서 tunnel website에 password에 입력한다.
streamlit run ${PWD}/app.py &> ${PWD}/logs.txt &
npx localtunnel --port 8501

FileNotFoundError: [Errno 2] No such file or directory: '${PWD}/logs.txt'